In [1]:
import os
from ultralytics import YOLO
from ultralytics.models.yolo.model import DetectionModel # Ensure this import is correct

In [2]:
data_path = os.path.join(os.getcwd(), 'datasets', 'split', '4', 'data.yaml')
test_data_path = os.path.join(os.getcwd(), 'datasets', 'split', '7', 'data.yaml')

In [3]:
# --- 3. Fine-Tuning ---
print("\n--- Starting Fine-Tuning ---")

# Load the model with transferred weights
model = YOLO('yolo11s.pt') # Load the result of the transfer

# --- Train Model ---
results = model.train(
    # ** Essential Paths & Config **
    data=str(data_path),                # Path to your dataset YAML file
    name='yolo11s',                 # Name for the training run directory
    exist_ok=False,                # Error if run name already exists
    save=True,                     # Save checkpoints and final model
    pretrained=True,
    
    # ** Requested Hyperparameters **
    epochs=100,                    # Number of training epochs
    batch=12,                       # Batch size
    imgsz=640,                     # Input image size (height=width=640)

    # ** Data Augmentation Control (ONLY MOSAIC ENABLED) **
    augment=False,                  # MUST be True to enable the augmentation pipeline for mosaic
)


--- Starting Fine-Tuning ---


100%|██████████| 18.4M/18.4M [00:01<00:00, 11.5MB/s]


New https://pypi.org/project/ultralytics/8.3.127 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080, 9877MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/data.yaml, epochs=100, time=None, patience=100, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo11s2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False

train: Scanning /home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/train/labels.cache... 10800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10800/10800 [00:00<?, ?it/s]
val: Scanning /home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/valid/labels.cache... 2700 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2700/2700 [00:00<?, ?it/s]


Plotting labels to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.00046875), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      3.57G     0.5617     0.2734       0.79        208        640: 100%|██████████| 900/900 [01:13<00:00, 12.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [00:06<00:00, 16.95it/s]

                   all       2700      50770      0.874      0.774      0.797      0.611



100 epochs completed in 2.287 hours.
Optimizer stripped from /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s2/weights/last.pt, 19.2MB
Optimizer stripped from /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s2/weights/best.pt, 19.2MB

Validating /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s2/weights/best.pt...
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080, 9877MiB)
YOLO11s summary (fused): 100 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [00:09<00:00, 12.32it/s]


                   all       2700      50770      0.874      0.774      0.797      0.611
                player       2700      43052      0.963      0.981       0.99      0.818
            goalkeeper       1302       1302      0.966      0.919      0.947       0.72
               referee       2517       4104      0.961      0.978      0.989      0.804
                  ball       2312       2312      0.604      0.219      0.262      0.103
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s2


In [5]:
# --- 4. Validation ---
print("\n--- Starting Validation ---")

print(f"Validating on Game ID 7: {test_data_path}")
try:
    res_flow = model.val(data=str(test_data_path)) # Use the trained model object
    print("Validation Results (Game ID 7):")
    # print(res_flow)
except Exception as e:
    print(f"Error during validation on {test_data_path}: {e}")


--- Starting Validation ---
Validating on Game ID 7: /home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/7/data.yaml
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080, 9877MiB)


val: Scanning /home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/7/test/labels.cache... 12000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12000/12000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1000/1000 [00:45<00:00, 21.91it/s]


                   all      12000     182533      0.557      0.421       0.49      0.278
                player      12000     152869       0.94      0.557      0.882      0.526
            goalkeeper       6637       6637     0.0681      0.247     0.0668      0.039
               referee      10373      13547      0.736      0.878      0.887      0.508
                  ball       9302       9480      0.483   0.000844      0.126     0.0412
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s23
Validation Results (Game ID 7):
